In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore',category=FutureWarning)

from keras.models import Sequential
from keras.layers import Dense

import keras as ks

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import PowerTransformer, MinMaxScaler, PolynomialFeatures
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.pipeline import Pipeline

import tensorflow as tf

from keras.callbacks import EarlyStopping, ModelCheckpoint

In [28]:
#tf.config.list_physical_devices('GPU')

In [29]:
#tf.config.experimental.list_physical_devices(device_type = None)

In [30]:
#tf.config.list_physical_devices(device_type = None)

In [31]:
with tf.device('/gpu:1'):
    train = pd.read_csv('data/train.csv')
    test = pd.read_csv('data/test.csv')

In [32]:
with tf.device('/gpu:1'):
    pd.set_option('display.max_columns', 500)

In [33]:
with tf.device('/gpu:1'):
    full_columns = train.columns.tolist()

In [34]:
with tf.device('/gpu:1'):
    full_columns.remove('permeability')

In [35]:
with tf.device('/gpu:1'):
    X = pd.DataFrame(train, columns = full_columns)

    y = train['permeability']

In [36]:
with tf.device('/gpu:1'):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state = 8669, test_size = 0.2
    )

In [37]:
with tf.device('/gpu:1'):
    nn = MLPRegressor(hidden_layer_sizes = (128,128,128,128), activation = 'relu', solver = 'adam', 
                  validation_fraction = 0.2, max_iter = 100000, early_stopping = True, 
                  random_state = 8669, shuffle = True, n_iter_no_change = 500, alpha = .0001)

In [38]:
with tf.device('/gpu:1'):
    ct = ColumnTransformer(transformers = [
        ('yeo-johnson', PowerTransformer(), full_columns),
    ], remainder = 'passthrough')

    pipe = Pipeline(steps = [
        ('transform', ct),
        ('scale', MinMaxScaler())
    ])

In [39]:
with tf.device('/gpu:1'):
    X_train = pipe.fit_transform(X_train)
    X_test = pipe.fit_transform(X_test)

#X_train = pipe.fit_transform(X_train)
#X_test = pipe.transform(X_test)

In [40]:
%%time

with tf.device('/gpu:1'):
    nn.fit(X_train, y_train)

Wall time: 11min 17s


In [41]:
with tf.device('/gpu:1'):
    print(mean_absolute_error(y_test, nn.predict(X_test)))

0.542188815609699


In [42]:
with tf.device('/gpu:1'):
    print(mean_squared_error(y_test, nn.predict(X_test)))

1.9406854173459418


In [43]:
with tf.device('/gpu:1'):
    print(np.sqrt(mean_squared_error(np.log(y_test + 1), np.log(nn.predict(X_test)+1))))

0.0723103711188141


In [17]:
with tf.device('/gpu:1'):
    test = pd.DataFrame(pipe.transform(test), columns = full_columns)

In [18]:
with tf.device('/gpu:1'):
    results = nn.predict(test)

In [44]:
with tf.device('/gpu:1'):
    print(nn.loss_)

0.25811720704353025


In [45]:
with tf.device('/gpu:1'):
    print(nn.n_iter_)

2215


In [46]:
with tf.device('/gpu:1'):
    print(nn.n_layers_)

6


In [22]:
with tf.device('/gpu:1'):
    results = pd.DataFrame(results)

In [23]:
with tf.device('/gpu:1'):
    test_results = pd.DataFrame()

In [24]:
with tf.device('/gpu:1'):
    test_results['permeability'] = results[0]

In [25]:
with tf.device('/gpu:1'):
    test_results.to_csv('data/nn/mlpr_results_21.csv', index_label = 'id')